# LUAD CPTAC analysis

In [1]:
import pandas as pd
import numpy as np
from msresist.pre_processing import preprocessing
from msresist.clustering import MassSpecClustering
import matplotlib.pyplot as plt

In [2]:
# pd.set_option('display.max_colwidth', 1000)
# pd.set_option('display.max_rows', 1000000)
pd.set_option('display.max_columns', 1000)

## 1 - Import and Preprocess Data

In [3]:
# X = preprocessing(CPTAC=True, log2T=True)
X = pd.read_csv("msresist/data/MS/CPTAC/CPTAC-preprocessedMotfis.csv")

d = X.select_dtypes(include=['float64']).T
i = X.select_dtypes(include=['object'])

Drop NaNs:

In [4]:
d = d.iloc[:8, :].dropna(axis=1)
i = i.iloc[d.columns, :]

Reduce data set:

In [5]:
# npept = 5000
# d = d.iloc[:, :npept]
# i = i.iloc[:npept, :]

In [6]:
distance_method = "Binomial"
ncl = 5
GMMweight = 0.0

MSC = MassSpecClustering(i, ncl, GMMweight=GMMweight, distance_method=distance_method, n_runs=1).fit(d, "NA")

run:  0
iter:  0
Re-initialize GMM clusters, empty cluster(s) at iteration 0

/usr/local/lib/python3.7/dist-packages/Bio/Seq.py:182: BiopythonWarning: Biopython Seq objects now use string comparison. Older versions of Biopython used object comparison. During this transition, please use hash(id(my_seq)) or my_dict[id(my_seq)] if you want the old behaviour, or use hash(str(my_seq)) or my_dict[str(my_seq)] for the new string hashing behaviour.
  BiopythonWarning,



iter:  1
-19.436879172212095
[[-1.33834526e+00 -9.34802642e-02 -1.26720358e-01 -1.41866635e-03
  -5.40314007e-02]
 [-1.02240121e+00 -1.73648032e-01 -1.33571507e-01 -2.51371049e-03
  -7.55613632e-02]
 [-1.47746298e+00 -1.66176259e-01 -2.44666721e-02 -3.57304109e-03
  -4.85354549e-02]
 ...
 [-3.07841588e-01 -3.24190598e-01 -3.09299879e-01 -1.06049206e-02
  -2.00350184e-01]
 [-3.02458360e-01 -3.66985925e-01 -2.49478690e-01 -2.83637925e-02
  -2.02189902e-01]
 [-8.07803319e-03 -6.48944151e-02 -5.63953043e-01 -1.85005246e-02
  -6.53438090e-01]]
[[-1.33834526e+00 -9.34802642e-02 -1.26720358e-01 -1.41866635e-03
  -5.40314007e-02]
 [-1.02240121e+00 -1.73648032e-01 -1.33571507e-01 -2.51371049e-03
  -7.55613632e-02]
 [-1.47746298e+00 -1.66176259e-01 -2.44666721e-02 -3.57304109e-03
  -4.85354549e-02]
 ...
 [-3.07841588e-01 -3.24190598e-01 -3.09299879e-01 -1.06049206e-02
  -2.00350184e-01]
 [-3.02458360e-01 -3.66985925e-01 -2.49478690e-01 -2.83637925e-02
  -2.02189902e-01]
 [-8.07803319e-03 -6.489

SystemExit: 

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
centers = MSC.transform(d)

In [ ]:
centers